# 2.2.1 创建Tensor

导入PyTorch

In [2]:
import torch

初始化5*3的未初始化的Tensor

In [9]:
x=torch.empty(5,3)
print(x)

tensor([[1.7448e-06, 4.5741e-41, 1.7448e-06],
        [4.5741e-41, 1.1210e-43, 0.0000e+00],
        [1.5695e-43, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 2.0755e-07, 4.5740e-41],
        [2.0752e-07, 4.5740e-41, 2.0753e-07]])


初始化5*3随机初始化的Tensor

In [4]:
x=torch.rand(5,3)
print(x)

tensor([[0.5766, 0.7708, 0.3185],
        [0.6744, 0.9545, 0.2274],
        [0.0468, 0.7664, 0.2858],
        [0.4112, 0.9296, 0.6073],
        [0.6964, 0.1233, 0.0416]])


创建5*3的long型全0的Tensor

In [5]:
x=torch.zeros(5,3,dtype=torch.long)
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


直接根据数据创建

In [12]:
x=torch.tensor([5.5,3])
print(x)

tensor([5.5000, 3.0000])


通过现有的Tensor来创建，此方法会默认重用输入Tensor的一些属性，例如数据类型，除非自定义数据类型。

In [17]:
x=x.new_ones(5,3,dtype=torch.float64)
print(x)
x=torch.rand_like(x, dtype=torch.float)
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([[0.8561, 0.4077, 0.1083],
        [0.5468, 0.3390, 0.3617],
        [0.0771, 0.4946, 0.8974],
        [0.4927, 0.8878, 0.2452],
        [0.3280, 0.7919, 0.4492]])


通过shape()或者size()来获取Tensor的形状

In [19]:
x.size()

torch.Size([5, 3])

In [21]:
x.shape

torch.Size([5, 3])

# 2.2.2 Tensor的基本操作

## 算术操作

### 加法形式1 +

In [22]:
y=torch.rand(5,3)
print(x+y)

tensor([[1.4285, 0.4263, 0.4656],
        [1.1913, 1.0028, 1.1205],
        [0.6339, 1.0509, 1.2761],
        [1.0600, 1.8412, 0.5689],
        [1.2611, 1.1846, 0.7598]])


### 加法形式2 add

In [24]:
torch.add(x,y)

tensor([[1.4285, 0.4263, 0.4656],
        [1.1913, 1.0028, 1.1205],
        [0.6339, 1.0509, 1.2761],
        [1.0600, 1.8412, 0.5689],
        [1.2611, 1.1846, 0.7598]])

指定输出

In [28]:
result=torch.empty(5, 3) 
torch.add(x, y, out=result)
print(result)

tensor([[1.4285, 0.4263, 0.4656],
        [1.1913, 1.0028, 1.1205],
        [0.6339, 1.0509, 1.2761],
        [1.0600, 1.8412, 0.5689],
        [1.2611, 1.1846, 0.7598]])


### 加法形式3 add_

__替换__ PyTorch操作inplace版本都有后缀_, 例如x.copy_(y), x.t_()

In [29]:
y.add_(x)
print(y)

tensor([[1.4285, 0.4263, 0.4656],
        [1.1913, 1.0028, 1.1205],
        [0.6339, 1.0509, 1.2761],
        [1.0600, 1.8412, 0.5689],
        [1.2611, 1.1846, 0.7598]])


## 索引

类似NumPy的索引操作来访问Tensor的一部分，需要注意的是：__索引出来的结果与原数据共享内存，也即修改一个，另一个会跟着修改。__

In [32]:
y=x[0,:]
y+=1
print(y)
print(x[0,:])

tensor([2.8561, 2.4077, 2.1083])
tensor([2.8561, 2.4077, 2.1083])


## 改变形状

view()返回的新Tensor与源Tensor虽然可能有不同的size，但是是__共享data__的，也即更改其中的一个，另外一个也会跟着改变。(顾名思义，view仅仅是改变了对这个张量的观察角度，内部数据并未改变)

In [34]:
y=x.view(15)
z=x.view(-1,5)
print(x.shape,y.shape,z.shape)

torch.Size([5, 3]) torch.Size([15]) torch.Size([3, 5])


如果想返回一个真正的副本（即不共享内存），reshape（）可以改变形状，但是此函数并不能一定保证返回的是其拷贝，所以不推荐使用。推荐先用__clone()__创造一个副本然后再使用__view()__

In [35]:
x_cp=x.clone().view(15)
x-=1
print(x)
print(x_cp)

tensor([[ 1.8561,  1.4077,  1.1083],
        [-0.4532, -0.6610, -0.6383],
        [-0.9229, -0.5054, -0.1026],
        [-0.5073, -0.1122, -0.7548],
        [-0.6720, -0.2081, -0.5508]])
tensor([2.8561, 2.4077, 2.1083, 0.5468, 0.3390, 0.3617, 0.0771, 0.4946, 0.8974,
        0.4927, 0.8878, 0.2452, 0.3280, 0.7919, 0.4492])


# 2.2.3 广播机制

对两个形状不同的Tensor按元素运算时，可能会触发广播（broadcasting）机制：先适当复制元素使这两个Tensor形状相同后再按元素运算。

In [38]:
x=torch.arange(1,3).view(1,2)
print(x)
y=torch.arange(1,4).view(3,1)
print(y)
print(x+y)

tensor([[1, 2]])
tensor([[1],
        [2],
        [3]])
tensor([[2, 3],
        [3, 4],
        [4, 5]])


# 2.2.4 运算的内存开销

索引操作是不会开辟新内存的，而像y = x + y这样的运算是会新开内存的，然后将y指向新内存。

In [39]:
x=torch.tensor([1,2])
y=torch.tensor([3,4])
id_before=id(y)
y=y+x
print(id(y),id_before)

140195047041792 140195048791040


如果想指定结果到原来的y的内存，我们可以使用索引来进行替换操作

In [40]:
x=torch.tensor([1,2])
y=torch.tensor([3,4])
id_before=id(y)
y[:]=y+x
print(id(y),id_before)

140195077338304 140195077338304


还可以使用运算符全名函数中的__out参数__、自加运算符__+=__（__add_()__）

In [42]:
x=torch.tensor([1,2])
y=torch.tensor([3,4])
id_before=id(y)
torch.add(x,y,out=y)
print(id(y),id_before)

140195044945472 140195044945472


In [43]:
x=torch.tensor([1,2])
y=torch.tensor([3,4])
id_before=id(y)
y+=x
print(id(y),id_before)

140195044942656 140195044942656


In [45]:
x=torch.tensor([1,2])
y=torch.tensor([3,4])
id_before=id(y)
y.add_(x)
print(id(y),id_before)

140197128459648 140197128459648


# 2.2.5 Tensor和Numpy相互转换

numpy() 和 from_numpy() 实现Tensor和Numpy相互转换（共享内存）

## Tenso转Numpy

In [49]:
a=torch.ones(5)
b=a.numpy()
print(a,b)

a+=1
print(a,b)

b+=1
print(a,b)

tensor([1., 1., 1., 1., 1.]) [1. 1. 1. 1. 1.]
tensor([2., 2., 2., 2., 2.]) [2. 2. 2. 2. 2.]
tensor([3., 3., 3., 3., 3.]) [3. 3. 3. 3. 3.]


## Numpy转Tensor

In [50]:
import numpy as np
a=np.ones(4)
b=torch.from_numpy(a)
print(a,b)

a+=1
print(a,b)

b+=1
print(a,b)

[1. 1. 1. 1.] tensor([1., 1., 1., 1.], dtype=torch.float64)
[2. 2. 2. 2.] tensor([2., 2., 2., 2.], dtype=torch.float64)
[3. 3. 3. 3.] tensor([3., 3., 3., 3.], dtype=torch.float64)


# 2.2.6 GPU

to()

In [53]:
device=torch.device('cuda')
y=torch.ones_like(x,device=device)
x=x.to(device)
z=x+y
print(z)
print(z.to('cpu',torch.double))

tensor([2, 3], device='cuda:0')
tensor([2., 3.], dtype=torch.float64)
